In [ ]:
!pip install datasets

In [ ]:
!python -m spacy init config config.cfg --pipeline ner --lang en

In [ ]:
import spacy
from spacy.tokens import DocBin
from datasets import load_dataset

In [ ]:
dataset = load_dataset("eriktks/conll2003")

train_dataset = dataset['train']
val_dataset = dataset['validation']
test_dataset = dataset['test']

print("Train size:", len(train_dataset))
print("Validation size:", len(val_dataset))
print("Test size:", len(test_dataset))

print(train_dataset[0])

In [ ]:
nlp = spacy.blank("en")

def convert_to_spacy(dataset, output_path):
    doc_bin = DocBin()
    for data in dataset:
        tokens = data["tokens"]
        tags = data["ner_tags"]
        doc = nlp.make_doc(" ".join(tokens))
        ents = []
        start = 0
        for token, label in zip(doc, tags):
            word = token.text
            end = start + len(word)
            if label != 0:
                label_str = dataset.features["ner_tags"].feature.names[label]
                if label_str.startswith("B-"):
                    ent_start = token.idx
                    ent_end = token.idx + len(token)
                    ent_label = label_str[2:]
                    ents.append(spacy.tokens.Span(doc, token.i, token.i + 1, label=ent_label))
            start += len(word) + 1
        doc.ents = ents
        doc_bin.add(doc)
    doc_bin.to_disk(output_path)

convert_to_spacy(train_dataset, "train.spacy")
convert_to_spacy(val_dataset, "dev.spacy")
convert_to_spacy(test_dataset, "test.spacy")

In [ ]:
!python -m spacy train config.cfg --output ./output --gpu-id 0

In [ ]:
!python -m spacy evaluate output/model-best test.spacy --gpu-id 0

### **Note:** Train and Evaluation outputs are in seperate file.